The purpose of this script is to look into which rosetta F22 images view a specified 
ground point of the CG67 (within a specific degree). This cell calculates the theta angle
between the selected ground point vector and the ground point vector for the center pixel
of an input image. Theta values are stored for all input images in a dictionary of format 
{absPath/imgName: theta}.

In [1]:
import numpy as np
import subprocess, pvl, os, json

## User Inputs
variables set for testing purposes

In [31]:
ax = 'x' #could be set to 'x' 'y' or 'z'
ori = 'neg' #could be set to positive 'pos' or negative 'neg'

#variables required for future module
gp_perspective = [-1, 0, 0] #x,y,z coordinates of the ground point perspective is centered around; typically set by user
img_file = 'F22_img.lis'
cexist = True #boolean indicating if the 

In [32]:
#read in entire input file and save as list
img_list = open(img_file).readlines()
img_dict = {} #initilize image:theta dictionary

#create sub directory for cubes
cur_dir = os.getcwd()
if not os.path.exists(os.path.join(cur_dir, 'cube')):
    os.makedirs(os.path.join(cur_dir, 'cube'))

In [33]:
unit_gp_perspective = gp_perspective / np.linalg.norm(gp_perspective)

for i in range(0,len(img_list)):
    try:
        img = img_list[i]
        image_basename = os.path.splitext(os.path.basename(img))[0]
        cube = os.path.join('cube', image_basename + '.cub')
        
        if cexist == False: #if cubes do not exist of the images you want to check
            #ingest image
            command = ['rososiris2isis',
                       'from={}'.format(img),
                       'to={}'.format(cube)]
            result = subprocess.run(
                    command,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    check=True)

            #spiceinit
            command = ['spiceinit',
                       'from={}'.format(cube),
                       'shape=user',
                       'model=$ISIS3DATA/rosetta/kernels/dsk/ROS_CG_M004_OSPGDLR_U_V1.bds',
                       '-preference=IsisPreferences_Bullet']
            result = subprocess.run(
                    command,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE,
                    check=True)

        #campt
        command = ['campt',
                   'from={}'.format(cube)]
        result = subprocess.run(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                check=True)
        cam_out = pvl.loads(result.stdout.decode())

        #calculate theta and compare to tolerance 
        gp_input = np.array(cam_out['GroundPoint']['BodyFixedCoordinate'][0]) #ground point vector of center pixel of input image
        unit_gp_input = gp_input / np.linalg.norm(gp_input) #unitize
        theta = np.arccos(np.dot(unit_gp_input, unit_gp_perspective))*180/np.pi #calc angle between perspective gp vector and images center pixel gp vector

        #create dictionary of filename: theta 
        img_dict.update({'{}'.format(img.strip()): theta})

        #print progress
        print('Image {} of {} complete'.format(i+1, len(img_list)))
        
    except subprocess.CalledProcessError as ex:
        print('Image {} failed on ISIS call:'.format(i+1))
        print(' '.join(ex.cmd))
        print('ISIS exception:')
        print(ex.stderr.decode())
        continue;
        
    except Exception as ex:
        print('Image {} failed with exception:'.format(i+1))
        print(ex)
        continue;


Image 1 of 10 complete
Image 2 of 10 complete
Image 3 of 10 complete
Image 4 of 10 complete
Image 5 of 10 complete
Image 6 of 10 complete
Image 7 of 10 complete
Image 8 of 10 complete
Image 9 of 10 complete
Image 10 of 10 complete


In [29]:
''''''
theta_tol = 45

img_valid = []
cnt = 0
for i in range(0,len(img_list)):
    try:
        img = img_list[i]
        image_basename = os.path.splitext(os.path.basename(img))[0]
        #if theta, from dictionary, is within tolerance
        if img_dict['{}'.format(img.strip())] < theta_tol:
            #save img base name to list for pricinple axis perspective
            img_valid.append('{}'.format(img_list[i].strip()))
            
    except KeyError:
        #continue
        cnt += 1
        #print('{} not spiceinited'.format(image_basename))
        
print('{} images out of {} were not spice inited in previous step'.format(cnt, len(img_list)))
print('{} images out of these remaining {} were valid for the {} principal axis perspective with a theta tolerance of {}'.format(len(img_valid), len(img_list)-cnt, ax+ori, theta_tol))        

169 images out of 358 were not spice inited in previous step
24 images out of these remaining 189 were valid for the xpos principal axis perspective with a theta tolerance of 45


In [30]:
#create and write a text file to list valid image names 
sub_dir = ax+ori
ofilename = ax+ori+'_valid_imgs.txt'
ofilepath = os.path.join(cur_dir,sub_dir,ofilename)
of = open(ofilepath,'w')
of.write('\n'.join(img_valid))
#pickle.dump(img_valid, of) #write the list of valid images to file
of.close()

#create and write a text file to use as output log image name: theta calculated
log_filename = ax+ori+'_log.txt'
log_filepath = os.path.join(cur_dir,sub_dir,log_filename)
lf = open(log_filepath,'w')
lf.write("\n".join("{}: theta = {}".format(k, v) for k, v in img_dict.items())) #write out the dictionary of img_names:theta to file
lf.close()